# Pytorch

In a previous workshop, you learned how to use Numpy to simply manage matrices in Python.
In order to make the creation of neural networks easier and faster, we will learn to use Pytorch, a library created by Meta AI.

## 1. Tensors

Pytorch makes it very easy to use the GPU to perform matrix calculations more quickly without having to write additional code.
Pytorch proposes an overlay to the Numpy arrays called Tensor. More simply, a Pytorch tensor is a Numpy matrix whose operations can be calculated on the GPU.

Let's create our first tensor !

In [ ]:
# Import PyTorch
import torch

# Create a Tensor filled with 0 (we can also create a tensor filled with 1 using torch.ones())
x = torch.zeros(3, 4)

print(x)
print(x.shape)

In [ ]:
# Create a Tensor filled with random values (between 0 and 1)
y = torch.rand(3, 4)

print(y)
print(y.shape)

In [ ]:
# We can apply arithmetics operations on tensors

print(y)
print(y + 1)
print(y * 3)

There are many other operations that can be applied to tensors but we won't need them here. You can find out more about tensors by following this link: [Pytorch Tensors](https://pytorch.org/tutorials/beginner/introyt/tensors_deeper_tutorial.html)